In [40]:
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.io as sp

from tqdm import tqdm
from scipy.linalg import eigh
from numpy.linalg import norm

class Group_dysco_analysis: 
    def __init__(self, datafolder, suffix=0): 
        # datafolder (str): absolute path to folder with ONLY subject files
        # suffix to choose load method (int): 0 = txt; 1 = mat
        self.datafolder = datafolder
        self.suffix = suffix

    def get_filename(self):
        files = [f for f in os.listdir(self.datafolder)]

        return files 

    def get_group_eigs(self, n_eigen, half_window_size): 
        # group_eigvec = (neigs x nroi x nf), group_eigval = (neigs x nroi x t x nf)
        group_eigvec = []
        group_eigval = []

        files = self.get_filename()
        if n_eigen > 2 * half_window_size:
            raise ValueError('Number of requested eigenvectors is too large')

        for f in files:      # need to do full absolute path 
            file = os.path.join(self.datafolder, f)
            if self.suffix == 1: 
                hold = sp.loadmat(file)
                timeseries = hold['timeSeries']
            else: 
                timeseries = np.genfromtxt(file)

            # calculating eigenvectors and eigenvectors for each subject     
            t, n = timeseries.shape #so here, t is the number of rows, n is the number of columns 
            total_iterations = t - 2 * half_window_size
            progress_bar_eigs = tqdm(total=total_iterations, desc="Calculating eigenvectors and eigenvalues:")
            eigenvectors = np.zeros((t - 2 * half_window_size, n, n_eigen)) #axes of eigenvector array 
            eigenvalues = np.zeros((n_eigen, t - 2 * half_window_size)) #axes of eigenvalue array 

            for i in range(t - 2 * half_window_size):
                truncated_timeseries = timeseries[i:i + 2 * half_window_size, :] #
                z_scored_truncated = (truncated_timeseries - np.mean(truncated_timeseries, axis=0)) / np.std(
                    truncated_timeseries, axis=0, ddof=1)

                normalizing_factor = z_scored_truncated.shape[0] - 1
                z_scored_truncated = (1 / np.sqrt(normalizing_factor)) * z_scored_truncated
                mini_matrix = z_scored_truncated @ z_scored_truncated.T
                ns = len(mini_matrix)

                eigenvalues_t, eigenvectors_t = eigh(mini_matrix, subset_by_index=[ns - n_eigen, ns - 1], overwrite_a=True,
                                                     check_finite=False)
                eigenvectors_t = np.flip(eigenvectors_t, axis=1)
                eigenvalues_t = np.flip(eigenvalues_t, axis=0)
                eigenvalues[:, i] = eigenvalues_t
                eigenvectors[i, :, :] = np.dot(z_scored_truncated.T, eigenvectors_t)

                for j in range(n_eigen):
                    eigenvectors[i, :, j] = eigenvectors[i, :, j] / np.linalg.norm(eigenvectors[i, :, j])
                    eigenvectors[i, :, j] = eigenvectors[i, :, j] * np.sqrt(eigenvalues[j, i])

                progress_bar_eigs.update(1)
                progress_bar_eigs.close()
                
            group_eigvec.append(eigenvectors)
            group_eigval.append(eigenvalues)

        return group_eigvec, group_eigval
    
    def get_group_norm_metastability(self, n_eigen, half_window_size, norm_type=2): 
        group_eigvec, group_eigval = self.get_group_eigs(n_eigen, half_window_size)
        
        group_norm = []
        group_metastability = []

        for index in range(0,(group_eigval.shape[2])):
            data = group_eigval[:, :, index]
            if norm_type == 1:
                norm = np.sum(np.abs(data),axis=0)
            elif norm_type == 2:
                norm = np.sqrt(np.sum(data) ** 2,axis=0)
            elif norm_type == np.inf:
                norm = np.max(data,axis=0)
            metastability =  np.std(norm)

            group_norm.append(norm)
            group_metastability.append(metastability)

        return group_norm, group_metastability
    
    





In [41]:
ketamine = Group_dysco_analysis('/Users/emiliewielezynski/Desktop/Ketamine_paper/michel_parcellated_data/keta/', 1)


In [43]:
group_eigvec, group_eigval = ketamine.get_group_eigs(10, 20)

Calculating eigenvectors and eigenvalues::   0%|          | 1/410 [00:00<00:27, 14.62it/s]


Calculating eigenvectors and eigenvalues::   0%|          | 1/410 [00:00<00:01, 239.36it/s]


In [ ]:


for index in range(0,(group_eigval.shape[2])):
            data = group_eigval[:, :, index]
            if norm_type == 1:
                norm = np.sum(np.abs(data),axis=0)
            elif norm_type == 2:
                norm = np.sqrt(np.sum(data) ** 2,axis=0)
            elif norm_type == np.inf:
                norm = np.max(data,axis=0)
            metastability =  np.std(norm)

            group_norm.append(norm)
            group_metastability.append(metastability)
